In [ ]:
from langchain.chat_models import ChatOpenAI

from langchain.memory import ConversationSummaryBufferMemory

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

from langchain.chains import LLMChain
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    temperature=0.1,
)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=100,
    return_messages=True,
    memory_key="history",
)

examples = [
    {
    "movie": "탑건을 이모티콘으로 바꿔줘",
    "answer": "🛩️👨‍✈️🔥"
    },
    {
    "movie": "대부를 이모티콘으로 바꿔줘",
    "answer": "👨‍👨‍👦🔫🍝"
    }
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{movie}"),
    ("ai", "{answer}")
])

prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples = examples
)

final_prompt = ChatPromptTemplate.from_messages([
    prompt,
    ("system", "당신은 영화 전문가입니다. 상대가 말하는 영화 제목을 듣고 해당 영화에 등장하는 핵심적인 요소 중 세 가지만 추려서 알맞은 이모티콘으로 나타냅니다. 상대방이 이모티콘으로 바꿔달라는 요청을 할 경우 대답은 반드시 세 개의 이모티콘으로만 이루어져 있어야 합니다. 그렇지 않은 상대방의 일반적인 질문에는 일반적으로 응답합니다."),
    MessagesPlaceholder(variable_name="history",),
    ("human", "{question}"),
    
])


def load_memory(_):
    
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(question):
    print("Before Memory:", memory.load_memory_variables({}))

    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )

    print("After Memory:", memory.load_memory_variables({}))

    print(result)

NameError: name 'load_dotenv' is not defined

In [124]:
invoke_chain("태극기휘날리며를 이모티콘으로 바꿔줘")

Before Memory: {'history': []}
After Memory: {'history': [HumanMessage(content='태극기휘날리며를 이모티콘으로 바꿔줘'), AIMessage(content='🇰🇷⚽️🏆')]}
content='🇰🇷⚽️🏆'


In [125]:
invoke_chain("독전을 이모티콘으로 바꿔줘")

Before Memory: {'history': [HumanMessage(content='태극기휘날리며를 이모티콘으로 바꿔줘'), AIMessage(content='🇰🇷⚽️🏆')]}
After Memory: {'history': [HumanMessage(content='태극기휘날리며를 이모티콘으로 바꿔줘'), AIMessage(content='🇰🇷⚽️🏆'), HumanMessage(content='독전을 이모티콘으로 바꿔줘'), AIMessage(content='🕵️\u200d♂️💼🔪')]}
content='🕵️\u200d♂️💼🔪'


In [126]:
invoke_chain("여태까지 어떤 영화를 이모티콘으로 바꿔달라고 요청했는지 리스트 형태로 보여줘")

Before Memory: {'history': [HumanMessage(content='태극기휘날리며를 이모티콘으로 바꿔줘'), AIMessage(content='🇰🇷⚽️🏆'), HumanMessage(content='독전을 이모티콘으로 바꿔줘'), AIMessage(content='🕵️\u200d♂️💼🔪')]}
After Memory: {'history': [SystemMessage(content='The human asks the AI to change the movie "Taegeukgi: The Brotherhood of War" into emojis. The AI responds with 🇰🇷⚽️🏆. The human then requests the movie "The Man from Nowhere" to be turned into emojis. The AI replies with 🕵️\u200d♂️💼🔪. Lastly, the human asks for a list of movies they have requested to be turned into emojis. The AI provides a list: 1. Top Gun: 🛩️👨\u200d✈️🔥 2. The Godfather: 👨\u200d👨\u200d👦🔫🍝 3. Taegeukgi: 🇰🇷⚽️🏆 4. The Man from Nowhere: 🕵️\u200d♂️💼🔪.')]}
content='1. 탑건: 🛩️👨\u200d✈️🔥\n2. 대부: 👨\u200d👨\u200d👦🔫🍝\n3. 태극기휘날리며: 🇰🇷⚽️🏆\n4. 독전: 🕵️\u200d♂️💼🔪'
